# Implementations

This notebook delves into the world of ECC implementations.

 - You will first [analyze traces](#Manual-analysis) collected from a Curve25519 implementation to learn information about the implementation.
    - How many iterations are in the scalar multiplier?
    - Is the scalar multiplier left-to-right or right-to-left?
 - Then you will explore the number of ECC implementations [combinatorially](#Implementation-space).

## Manual analysis

[sca25519](https://github.com/sca-secure-library-sca25519/sca25519) is an open-source implementation of the Curve25519 key-exchange (XDH) for the ARM Cortex-M4. It contains three **implementations**:

 - unprotected
 - ephemeral
 - static

that contain different countermeasures, with the unprotected one being free of any.

You will work with three trace sets of scalar multiplication:
 - **A**: [Download](https://neuromancer.sk/static/traces_A.pickle) Traces of the full scalar multiplication. 10 traces per implementation: all implementations. Fixed scalar and point.
 - **B**: [Download](https://neuromancer.sk/static/traces_B.pickle) Traces of the beginning of the scalar multiplication. 1000 traces. Unprotected implementation. Random scalar.
 - **C**: [Download](https://neuromancer.sk/static/traces_C.pickle) Traces of the end of the scalar multiplication. 1000 traces. Unprotected implementation. Random scalar. 

In [ ]:
from pyecsca.sca.trace_set import PickleTraceSet, HDF5TraceSet
from pyecsca.sca.trace import Trace
from pyecsca.sca.trace.plot import plot_trace, plot_traces
from pyecsca.ec.params import get_params
from pyecsca.ec.point import Point
from pyecsca.ec.mod import Mod

import holoviews as hv
import numpy as np

In [ ]:
hv.extension("bokeh")
%opts RGB [height=600, responsive=True]

In [ ]:
curve25519 = get_params("other", "Curve25519", "xz")
p = curve25519.curve.prime
n = curve25519.order

### <span style="color:#00468C; font-weight: bold;">Exercise</span>

Use trace set **A** and visually analyze the traces:

 - Plot them.
 - Compare them between implementations.
 - Process them using e.g. rolling mean and count peaks on them.
 - How many iterations does the scalar multiplier have in the unprotected case?

**Docs**<br/>
[plot module](https://neuromancer.sk/pyecsca/api/pyecsca.sca.trace.plot.html)<br/>
[scipy.signal.find_peaks](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.find_peaks.html)

In [ ]:
from pyecsca.sca.trace.plot import plot_trace, plot_traces
from pyecsca.sca.trace.process import rolling_mean
from scipy.signal import find_peaks

def plot_trace_peaks(peaks, trace):
    return plot_trace(trace) * hv.Points((peaks, trace.samples[peaks])).opts(color="red")

In [ ]:
traces_A = PickleTraceSet.read("../traces_A.pickle")

In [ ]:
traces_A[0]

In [ ]:
# Task: Plot two traces from the unprotected implementation.
# Hint: Look at the .meta attribute of the traces.
plot_traces(traces_A[0], traces_A[1])

In [ ]:
# Task: Plot traces from the other implementations.
plot_traces(traces_A[10])

In [ ]:
plot_traces(traces_A[20])

In [ ]:
# Task: Use rolling_mean and find_peaks to count the iterations in the unprotected implementation
# Note: Before applying the rolling mean, make sure to transform the dtype of the trace by doing trace.astype(np.float32) and using the result.
# Hint: You can use plot_trace_peaks to plot the trace along with the detected peaks from find_peaks.
meany = rolling_mean(traces_A[0].astype(np.float32), 5000)
peaks, other = find_peaks(meany.samples, height=10, distance=5000)
print(len(peaks))
plot_trace_peaks(peaks, meany)

### <span style="color:#00468C; font-weight: bold;">Exercise</span>

Use trace set **B** and correlate the most significant bits of the scalar with the trace set.

 - Is the multiplier left-to-right or right-to-left?

> Note: The correlation computation takes a fair bit of memory (4-8GB), make sure you have it. **This will likely not work on myBinder.**


**Docs**<br/>
[StackedTraces](https://neuromancer.sk/pyecsca/api/pyecsca.sca.stacked_traces.stacked_traces.html#pyecsca.sca.stacked_traces.stacked_traces.StackedTraces)<br/>
[CPUTraceManager.pearson_corr](https://neuromancer.sk/pyecsca/api/pyecsca.sca.stacked_traces.combine.html#pyecsca.sca.stacked_traces.combine.CPUTraceManager.pearson_corr)<br/>
[Leakage models, Hamming Weight, Bit](https://neuromancer.sk/pyecsca/api/pyecsca.sca.attack.leakage_model.html)

In [ ]:
from pyecsca.sca.stacked_traces import StackedTraces, CPUTraceManager
from pyecsca.sca.attack.leakage_model import HammingWeight, Bit
from pyecsca.sca.trace.plot import plot_trace, plot_traces
from pyecsca.sca.trace.process import normalize

In [ ]:
traces_B = PickleTraceSet.read("../traces_B.pickle")
traces_C = PickleTraceSet.read("../traces_C.pickle")

In [ ]:
stacked_B = StackedTraces.fromtraceset(traces_B)
mng_B = CPUTraceManager(stacked_B)

In [ ]:
corrs = []
for i in (253, 252, 251):
    lm = Bit(i)
    ivs = np.array(list(map(lambda t: lm(t.meta["scalar"]), traces_B)))
    corrs.append(normalize(Trace(mng_B.pearson_corr(ivs))))

In [ ]:
plot_traces(rolling_mean(corrs[0], 7000), rolling_mean(corrs[1], 7000), rolling_mean(corrs[2], 7000))

### <span style="color:#00468C; font-weight: bold;">Exercise</span>

Use trace set **C** and correlate Hamming Weights of a few bytes of the result of the scalar multiplication with the traces.

 - Where do you see correlation? What does it mean? Does it happen during the scalar multiplication?

This trace set comes from the same set as trace **B** but we aligned the traces at the end for you so you are able to detect the result leakage using correlation. 

**Docs**<br/>
[StackedTraces](https://neuromancer.sk/pyecsca/api/pyecsca.sca.stacked_traces.stacked_traces.html#pyecsca.sca.stacked_traces.stacked_traces.StackedTraces)<br/>
[CPUTraceManager.pearson_corr](https://neuromancer.sk/pyecsca/api/pyecsca.sca.stacked_traces.combine.html#pyecsca.sca.stacked_traces.combine.CPUTraceManager.pearson_corr)<br/>
[Leakage models, Hamming Weight, Bit](https://neuromancer.sk/pyecsca/api/pyecsca.sca.attack.leakage_model.html)

In [ ]:
#In the original traces correlate the traces with the hamming wieght of data at index 70
#In your traces it will be HW of one byte of the point
HW = [bin(n).count("1") for n in range(0, 256)]

### <span style="color:#00468C; font-weight: bold;">Exercise</span>

Use trace set **B** and to perform a CPA-like attack on first 1000 traces. 

Let's veryify that the traces corresponds to X25519 where the most significant bit is always 1. 

**Docs**<br/>

In [ ]:
#ECC code:
curve25519_P = 2 ** 255 - 19
curve25519_A = 486662
# from https://gist.github.com/nickovs/cc3c22d15f239a2640c185035c06f8a3
def _point_add(point_n, point_m, point_diff):
    """Given the projection of two points and their difference, return their sum"""
    (xn, zn) = point_n
    (xm, zm) = point_m
    (x_diff, z_diff) = point_diff
    x = (z_diff << 2) * (xm * xn - zm * zn) ** 2
    z = (x_diff << 2) * (xm * zn - zm * xn) ** 2
    return x % curve25519_P, z % curve25519_P

# from https://gist.github.com/nickovs/cc3c22d15f239a2640c185035c06f8a3
def _point_double(point_n):
    """Double a point provided in projective coordinates"""
    (xn, zn) = point_n
    xn2 = xn ** 2
    zn2 = zn ** 2
    x = (xn2 - zn2) ** 2
    xzn = xn * zn
    z = 4 * xzn * (xn2 + curve25519_A * xzn + zn2)
    return x % curve25519_P, z % curve25519_P


# from https://gist.github.com/nickovs/cc3c22d15f239a2640c185035c06f8a3
def _const_time_swap(a, b, swap):
    """Swap two values in constant time"""
    index = int(swap) * 2
    temp = (a, b, b, a)
    return temp[index:index+2]


# from https://gist.github.com/nickovs/cc3c22d15f239a2640c185035c06f8a3
def _raw_curve25519(base, n):
    """Raise the point base to the power n"""
    zero = (1, 0)
    one = (base, 1)
    mP, m1P = zero, one

    for i in reversed(range(256)):
        bit = bool(n & (1 << i))
        mP, m1P = _const_time_swap(mP, m1P, bit)

        if i >= 247:
            print("x1: ", hex(mP[0]))
            print("z1: ", hex(mP[1]))
            print("x2: ", hex(m1P[0]))
            print("z2: ", hex(m1P[1]))
            print("--------------------")

        mP, m1P = _point_double(mP), _point_add(mP, m1P, one)
        mP, m1P = _const_time_swap(mP, m1P, bit)

    x, z = mP
    x1, z1 = m1P
    #inv_z = pow(z, curve25519_P - 2, curve25519_P)
    #return (x * inv_z) % curve25519_P
    return x, z, x1, z1

# function taken from https://github.com/lhrbacek/ed25519-py/blob/main/curve25519.py
# from https://gist.github.com/nickovs/cc3c22d15f239a2640c185035c06f8a3
# base - x coordinate of point as bytearray/bytes in big endian (msb on left)
# n - scalar
#
# Usage:
#    point = bytearray(bytes.fromhex('010203'))
#    g = scamult_once_curve25519(point, 1)
#    print(hex(g))
#
def scamult_once_curve25519(base, n):
    """Raise the point base to the power n, return x of mP without conoversion back to affine"""
    Px = int.from_bytes(base, byteorder='little')
    zero = (1, 0)
    one = (Px, 1)
    mP, m1P = zero, one

    for i in reversed(range(1)):
        bit = bool(n & (1 << i))
        mP, m1P = _const_time_swap(mP, m1P, bit)
        mP, m1P = _point_double(mP), _point_add(mP, m1P, one)
        mP, m1P = _const_time_swap(mP, m1P, bit)

    x, z = mP
    x1, z1 = m1P
    #inv_z = pow(z, curve25519_P - 2, curve25519_P)
    #return (x * inv_z) % curve25519_P
    return x

hex_string = "a9849099874a6df03299eb341e7fd9be4d8e922ecf5d039a01d534c505b36483"
bytes_result = bytearray.fromhex(hex_string)
print(bytes_result)

point = bytearray(bytes.fromhex(hex_string))
point[31]=point[31]&0x7F
g = scamult_once_curve25519(point, 1)
print(hex(g))
byte_representation = g.to_bytes(32, byteorder='big')
print(''.join(format(x, '02x') for x in byte_representation))

## Implementation space
Let's explore the space of ECC implementations and examine how large and diverse it is. Recall the slide about real-world ECC implementations in open-source libraries and how varied those were. Now, we can use **pyecsca** to enumerate ECC implementation configurations.

> Note: These cells are prepared for you, you do not need to implement anything.

An ECC implementation configuration in **pyecsca** has the following attributes:

In [ ]:
from typing import get_args
from IPython.display import display, HTML
from pyecsca.ec.configuration import Configuration
from dataclasses import fields

content = ""
for field in fields(Configuration):
    name = field.name
    tp = field.type
    doc = tp.__doc__
    if get_args(field.type):
            doc = get_args(field.type)[0].__doc__
    tn = repr(tp) if tp.__module__ == "typing" else f"{tp.__module__}.{tp.__name__}"
    entry = ""
    entry += f"<b>{name}</b> <code>{tn}</code><br/>"
    entry += f"<pre>{doc}</pre><br/>"
    if hasattr(tp, "names"):
        for enum_name in tp.names():
            entry += f"{enum_name}<br/>"
    content += f"<li>{entry}</li>"
display(HTML(f"<ul>{content}</ul>"))

The possible configurations can be generated using the `all_configurations()` function. The whole space of configurations is quite huge, so we will not enumerate them.

A large part of the configuration space is due to the independent options which consist of:
 
 - `hash_type` of type `HashType` $*6$
 - `mod_rand` of type `RandomMod` $*2$
 - `mult` of type `Multiplication` $*4$
 - `sqr` of type `Squaring` $*4$
 - `red` of type `Reduction` $*3$
 - `inv` of type `Inversion` $*2$

Without these, the space is somewhat smaller.

To restrict the generated configurations, pass keyword arguments to the
`all_configurations` matching the names of the attributes of the `Configuration` object.

Below, we look at how many configurations there are for projective coordinates on short-Weierstrass curves using
the left-to-right double-and-add scalar multiplier (which has several parametrizations itself).

In [ ]:
from pyecsca.ec.configuration import all_configurations, HashType, RandomMod, Multiplication, Squaring, Reduction, Inversion
from pyecsca.ec.model import ShortWeierstrassModel
from pyecsca.ec.mult import LTRMultiplier

model = ShortWeierstrassModel()
coords = model.coordinates["projective"]
scalarmult = LTRMultiplier
independent_opts = {
    "hash_type": HashType.SHA256,
    "mod_rand": RandomMod.SAMPLE,
    "mult": Multiplication.KARATSUBA,
    "sqr": Squaring.KARATSUBA,
    "red": Reduction.MONTGOMERY,
    "inv": Inversion.GCD
}

configs = list(all_configurations(model=model, coords=coords, scalarmult=scalarmult,
                                                              **independent_opts))
print(len(configs))

We see that when we fixed all parameters except for the scalar multiplier arguments 
(see the `LTRMultiplier` constructor) we obtained 1280 configurations. That number expresses all of the possible ways to use addition formulas for the `projective` coordinate system in the binary left-to-right multiplier as well as internal options of that multiplier:

 - whether it is "complete" in a sense that it starts processing at a constant bit (the bit-length od the order)
 - whether it is "double-and-add-always"
 - whether it "short-circuits" the formulas, i.e. detects that an exceptional point was input into them and returns correctly
   without executing them.

If we also restrict the formulas to concrete ones, we just get the variants of the scalar multiplier.

In [ ]:
formulas = {coords.formulas["add-2007-bl"], coords.formulas["dbl-2007-bl"]}

configs = list(filter(lambda cfg: set(cfg.scalarmult.formulas.values()) == formulas, configs))
print(len(configs))
for cfg in configs:
    print(str(cfg.scalarmult), "\n")

### Models
We can explore the number of configurations for each of the supported curve models.

In [ ]:
from IPython.display import HTML, display
import tabulate
from pyecsca.ec.model import *
from pyecsca.ec.mult import ProcessingDirection, AccumulationOrder

no_indep = (6*2*4*4*3*2)
no_ff = (6*2)

model_counts = [["Model", "All", "Without independent options", "Without independent options and scaling", "Without independent options and scalarmult options"]]
totals = ["Total", 0, 0, 0, 0]
for model in (ShortWeierstrassModel(), MontgomeryModel(), EdwardsModel(), TwistedEdwardsModel()):
    name = model.__class__.__name__
    count = sum(1 for _ in all_configurations(model=model, **independent_opts))
    count_no_scl = sum(1 for _ in all_configurations(model=model, **independent_opts, scalarmult={"scl": None}))
    count_no_opts = sum(1 for _ in all_configurations(model=model, **independent_opts, scalarmult={"scl": None, "always": True, "short_circuit": True, "complete": False, "precompute_negation": True, "width": 3, "m": 3, "direction": ProcessingDirection.LTR, "accumulation_order": AccumulationOrder.PeqPR, "recoding_direction": ProcessingDirection.LTR}))
    model_counts.append([name, count * no_ff, count, count_no_scl, count_no_opts])
    totals[1] += count * no_ff
    totals[2] += count
    totals[3] += count_no_scl
    totals[4] += count_no_opts
model_counts.append(totals)
display(HTML(tabulate.tabulate(model_counts, tablefmt="html", headers="firstrow")))

### Coordinate models

Let's now look at the configuration split for coordinate systems:

In [ ]:
coords_counts = [["Model", "Coords", "All", "Without independent options", "Without independent options and scaling", "Without independent options and scalarmult options"]]
for model in (ShortWeierstrassModel(), MontgomeryModel(), EdwardsModel(), TwistedEdwardsModel()):
    model_name = model.__class__.__name__
    coords_counts.append([model_name, "", "", "", "", ""])
    for coords in sorted(model.coordinates.values(), key=lambda c: c.name):
            coords_name = coords.name
            count = sum(1 for _ in all_configurations(model=model, coords=coords, **independent_opts))
            count_no_scl = sum(1 for _ in all_configurations(model=model, coords=coords, **independent_opts, scalarmult={"scl": None}))
            count_no_opts = sum(1 for _ in all_configurations(model=model, coords=coords, **independent_opts, scalarmult={"scl": None, "always": True, "short_circuit": True, "complete": False, "precompute_negation": True, "width": 3, "m": 3, "direction": ProcessingDirection.LTR, "accumulation_order": AccumulationOrder.PeqPR, "recoding_direction": ProcessingDirection.LTR}))
            coords_counts.append(["", coords_name, count * no_ff, count, count_no_scl, count_no_opts])

display(HTML(tabulate.tabulate(coords_counts, tablefmt="html", headers="firstrow")))

### Scalar multipliers

Finally, lets look at how the configurations look from the perspective of scalar multipliers.

In [ ]:
from pyecsca.ec.mult import ScalarMultiplier

def leaf_subclasses(cls):
    subs = cls.__subclasses__()
    result = set()
    for subclass in subs:
        if subclass.__subclasses__():
            result.update(leaf_subclasses(subclass))
        else:
            result.add(subclass)
    return result

mult_counts = [["ScalarMultiplier", "All", "Without independent options", "Without independent options and scaling", "Without independent options and scalarmult options"]]
for mult_cls in leaf_subclasses(ScalarMultiplier):
    count = sum(1 for _ in all_configurations(**independent_opts, scalarmult=mult_cls))
    count_no_scl = sum(1 for _ in all_configurations(**independent_opts, scalarmult={"cls": mult_cls, "scl": None}))
    count_no_opts = sum(1 for _ in all_configurations(**independent_opts, scalarmult={"cls": mult_cls, "scl": None, "always": True, "short_circuit": True, "complete": False, "precompute_negation": True, "width": 3, "m": 3, "direction": ProcessingDirection.LTR, "accumulation_order": AccumulationOrder.PeqPR, "recoding_direction": ProcessingDirection.LTR}))
    mult_counts.append([mult_cls.__name__, count * no_ff, count, count_no_scl, count_no_opts])

display(HTML(tabulate.tabulate(mult_counts, tablefmt="html", headers="firstrow")))